# Importa librerías

In [1]:
# Import libraries
import os
import requests
import pandas as pd
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.orchestration.metadata import sqlite_metadata_connection_config
from tensorflow_metadata.proto.v0 import statistics_pb2, schema_pb2
import tensorflow_data_validation as tfdv
from sklearn.model_selection import train_test_split
from tfx.types import standard_artifacts
from tfx.types import Channel
from tfx.v1.components import ImportSchemaGen
from tfx.types.standard_artifacts import Examples
from tfx.types import artifact_utils
from tfx.v1.components import ExampleValidator
from tfx.components import Transform
import ml_metadata as mlmd
from ml_metadata.metadata_store import metadata_store
from ml_metadata.proto import metadata_store_pb2


print('TF version:', tf.__version__)
print('TFDV version:', tfdv.version.__version__)

2025-03-03 03:44:14.220307: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-03 03:44:14.284758: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-03 03:44:14.284837: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-03 03:44:14.286748: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-03 03:44:14.295886: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-03 03:44:14.296981: I tensorflow/core/platform/cpu_feature_guard.cc:1

TF version: 2.15.1
TFDV version: 1.15.1


In [2]:
from data_preparation import *
from model_creation import *
from sklearn.feature_selection import SelectKBest, chi2, f_classif

# 2.1 Lectura de datos

In [3]:
## download the dataset
# Directory of the raw data files
_data_root = './data/covertype'
# Path to the raw training data
_data_filepath = os.path.join(_data_root, 'covertype_train.csv')
# Download data
os.makedirs(_data_root, exist_ok=True)
if not os.path.isfile(_data_filepath):
    #https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/
    url = 'https://docs.google.com/uc?export= \
    download&confirm={{VALUE}}&id=1lVF1BCWLH4eXXV_YOJzjR7xZjj-wAGj9'
    r = requests.get(url, allow_redirects=True, stream=True)
    open(_data_filepath, 'wb').write(r.content)
    
df = cargar_datos(_data_filepath)

Elevation                              int64
Aspect                                 int64
Slope                                  int64
Horizontal_Distance_To_Hydrology       int64
Vertical_Distance_To_Hydrology         int64
Horizontal_Distance_To_Roadways        int64
Hillshade_9am                          int64
Hillshade_Noon                         int64
Hillshade_3pm                          int64
Horizontal_Distance_To_Fire_Points     int64
Wilderness_Area                       object
Soil_Type                             object
Cover_Type                             int64
dtype: object


In [4]:
# Se seleccionan solo las variables numéricas
df_numerico = df.select_dtypes(include=['number'])

In [5]:
df_numerico.dtypes

Elevation                             int64
Aspect                                int64
Slope                                 int64
Horizontal_Distance_To_Hydrology      int64
Vertical_Distance_To_Hydrology        int64
Horizontal_Distance_To_Roadways       int64
Hillshade_9am                         int64
Hillshade_Noon                        int64
Hillshade_3pm                         int64
Horizontal_Distance_To_Fire_Points    int64
Cover_Type                            int64
dtype: object

In [6]:
X = df_numerico.drop(['Cover_Type'], axis=1)
y = df_numerico['Cover_Type']

# 3. Selección de características

In [7]:
# Aplicar SelectKBest
selector = SelectKBest(f_classif, k=8)
X_new = selector.fit_transform(X, y)

# Obtener los nombres de las columnas seleccionadas
columnas_seleccionadas = X.columns[selector.get_support()]

# Convertir nuevamente en DataFrame
X_new = pd.DataFrame(X_new, columns=columnas_seleccionadas, index=X.index)
df_new = pd.concat([X_new, y], axis=1)

In [8]:
# Se guardan los datos en ruta definida
root_path = './data/data_new'
file_path = os.path.join(root_path,"cover_new.csv")
df_new.to_csv(file_path)

# 4. Data Pipeline

## 4.1. Configurar el contexto interactivo


In [9]:
# Define un directorio persistente para los artefactos del pipeline
pipeline_root = "/ruta/persistente/para/pipeline_outputs"

# Inicia el contexto interactivo con la ruta definida
context = InteractiveContext(pipeline_root=pipeline_root)

# Define una ruta para la base de datos SQLite
metadata_path = "/ruta/persistente/para/metadata.sqlite"

# Configura la conexión a la metadata
metadata_config = sqlite_metadata_connection_config(metadata_path)

# Inicia el contexto con metadata persistente
context = InteractiveContext(pipeline_root=pipeline_root, metadata_connection_config=metadata_config)


## 4.2. Generando Ejemplos

In [10]:
# Crear el componente ExampleGen
example_gen = CsvExampleGen(input_base=root_path)
context.run(example_gen)



ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 8
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

## 4.3. Estadísticas


In [11]:
# Crear el componente StatisticsGen
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 9
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [12]:
stats_path = os.path.join(statistics_gen.outputs['statistics'].get()[0].uri,'Split-train','FeatureStats.pb')

# Leer el archivo en formato protobuf
with open(stats_path, "rb") as f:
    stats_proto = statistics_pb2.DatasetFeatureStatisticsList()
    stats_proto.ParseFromString(f.read())

# Visualizar estadísticas
tfdv.visualize_statistics(stats_proto)

## 4.4. Inferir el esquema


In [13]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'])
context.run(schema_gen)


ExecutionResult(
    component_id: SchemaGen
    execution_id: 10
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

## 4.5. Curando el esquema


In [14]:
# Obtener la URI del esquema generado por SchemaGen
schema_uri = schema_gen.outputs['schema'].get()[0].uri

# Cargar el esquema en formato protobuf
schema = tfdv.load_schema_text(schema_uri + "/schema.pbtxt")
# Definir rangos para variables numéricas
tfdv.set_domain(schema, "Hillshade_9am", schema_pb2.IntDomain(min=0, max=255))
tfdv.set_domain(schema, "Hillshade_Noon", schema_pb2.IntDomain(min=0, max=255))
tfdv.set_domain(schema, "Slope", schema_pb2.IntDomain(min=0, max=90))

# Definir Cover_Type como categórica
tfdv.set_domain(schema, "Cover_Type", schema_pb2.IntDomain(min=0, max=6, is_categorical = True))


## 4.6. Entornos de esquema


In [15]:
# Seleccionar un subconjunto aleatorio (por ejemplo, el 20% de los datos)
df_service = df_new.sample(frac=0.2, random_state=42).copy()

# Eliminar la columna de la etiqueta (Cover_Type)
df_service.drop(columns=["Cover_Type"], inplace=True)

service_path = "./data/data_service"
file_service_path = os.path.join(service_path,'service_data.csv')
# Guardar el conjunto de datos de servicio
df_service.to_csv(file_service_path)

In [16]:
# Se valida que sin entorno se marca la columna Cover_Type
options = tfdv.StatsOptions(schema=schema, infer_type_from_schema=True)
serving_stats = tfdv.generate_statistics_from_csv(file_service_path, stats_options=options)
serving_anomalies = tfdv.validate_statistics(serving_stats, schema)

tfdv.display_anomalies(serving_anomalies)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


,Anomaly short description,Anomaly long description
Feature name,,
'Cover_Type',Column dropped,Column is completely missing


In [17]:
# Se utilizan los entornos
# All features are by default in both TRAINING and SERVING environments.
schema.default_environment.append('TRAINING')
schema.default_environment.append('SERVING')

# Specify that 'Cover_Type' feature is not in SERVING environment.
tfdv.get_feature(schema, 'Cover_Type').not_in_environment.append('SERVING')
serving_anomalies_with_env = tfdv.validate_statistics(serving_stats, schema, environment='SERVING')

tfdv.display_anomalies(serving_anomalies_with_env)

In [18]:
# Guardo el esquema
schema_file = 'schema.pb'
schema_folder = './schema'
schema_path = os.path.join(schema_folder,schema_file)
tfdv.write_schema_text(schema, schema_path)

In [19]:
# Muestro el esquema y los cambios que se introducieron
schema = tfdv.load_schema_text(schema_path)  # Para el archivo de texto

In [20]:
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'',INT,required,,-
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90


In [21]:
schema.default_environment

['TRAINING', 'SERVING']

## 4.7. Entornos de esquema

In [22]:
schema_path = "./schema/schema.pb"

import_schema = ImportSchemaGen(schema_file=schema_path)
context.run(import_schema)

ExecutionResult(
    component_id: ImportSchemaGen
    execution_id: 11
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=ImportSchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [23]:
schema_channel = import_schema.outputs['schema']
# Crear el componente StatisticsGen
statistics_gen_actual = StatisticsGen(examples=example_gen.outputs['examples'],schema= schema_channel)
context.run(statistics_gen_actual)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 12
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [24]:
stats_path_actual = os.path.join(statistics_gen_actual.outputs['statistics'].get()[0].uri,'Split-train','FeatureStats.pb')

# Leer el archivo en formato protobuf
with open(stats_path_actual, "rb") as f:
    stats_proto_actual = statistics_pb2.DatasetFeatureStatisticsList()
    stats_proto_actual.ParseFromString(f.read())

# Visualizar estadísticas
tfdv.visualize_statistics(stats_proto_actual)

## 4.8. Comprobar anomalias
# Verificar anomalías usando ExampleValidator

In [25]:
example_validator = ExampleValidator(
    statistics=statistics_gen_actual.outputs["statistics"],
    schema= schema_channel
)
context.run(example_validator)
context.show(example_validator.outputs["anomalies"])

## 4.9. Ingeniería de caracteristicas y  4.10. Función de preprocesamiento

In [26]:
preprocessing_code = """
import tensorflow as tf
import tensorflow_transform as tft

# Declaración de constantes para características numéricas y categóricas.
NUMERIC_FEATURES = [
    'Elevation', 'Slope',
    'Horizontal_Distance_To_Hydrology',
    'Vertical_Distance_To_Hydrology',
    'Horizontal_Distance_To_Roadways',
    'Hillshade_9am', 'Hillshade_Noon',
    'Horizontal_Distance_To_Fire_Points'
]
# Llave para la etiqueta
LABEL_KEY = 'Cover_Type'

def preprocessing_fn(inputs):
    outputs = {}
    # Transformaciones para características numéricas:
    # Las primeras tres características se escalan a un rango [0, 1].
    for key in NUMERIC_FEATURES[:3]:
        outputs[key + '_scaled_0_1'] = _fillna(tft.scale_to_0_1(
            tf.cast(inputs[key], tf.float32)
        ))
    # Las siguientes tres características se escalan usando min-max scaling.
    for key in NUMERIC_FEATURES[3:6]:
        outputs[key + '_scaled_minmax'] = _fillna(tft.scale_by_min_max(
            tf.cast(inputs[key], tf.float32)
        ))
    # Las restantes se escalan usando z-score normalization.
    for key in NUMERIC_FEATURES[6:]:
        outputs[key + '_scaled_zscore'] = _fillna(tft.scale_to_z_score(
            tf.cast(inputs[key], tf.float32)
        ))
    
    # La etiqueta se mantiene sin transformación.
    outputs[LABEL_KEY] = inputs[LABEL_KEY]
    return outputs

def _fillna(t, value=0):
    if not isinstance(t, tf.sparse.SparseTensor):
        return t
    return tf.squeeze(tf.sparse.to_dense(
        tf.SparseTensor(t.indices, t.values, [t.dense_shape[0], 1]),
        default_value=value), axis=1)


"""

# Guardar
with open("preprocessing.py", "w") as f:
    f.write(preprocessing_code)



## 4.11. Transformar

In [27]:
from tfx.components import SchemaGen, StatisticsGen
from tfx.types import channel_utils
from tfx.types.standard_artifacts import Schema


# Ahora definir la ruta del módulo de preprocesamiento
module_file_path = "./preprocessing.py"


transform = Transform( examples= example_gen.outputs['examples'], 
            schema= schema_channel, 
            module_file= module_file_path, 
) 

context.run(transform, enable_cache= False)

running bdist_wheel
running build
running build_py
creating build/lib
copying data_preparation.py -> build/lib
copying model_creation.py -> build/lib
copying preprocessing.py -> build/lib
installing to /tmp/tmp63jpal_d
running install
running install_lib
copying build/lib/data_preparation.py -> /tmp/tmp63jpal_d/.
copying build/lib/model_creation.py -> /tmp/tmp63jpal_d/.
copying build/lib/preprocessing.py -> /tmp/tmp63jpal_d/.
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'


/opt/venv/lib/python3.9/site-packages/setuptools/_distutils/cmd.py:79: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmp63jpal_d/./tfx_user_code_Transform-0.0+72cadd684014286d19d8fc3ce643d86698e6d114331192894dd80b7d1c8544e3-py3.9.egg-info
running install_scripts
creating /tmp/tmp63jpal_d/tfx_user_code_transform-0.0+72cadd684014286d19d8fc3ce643d86698e6d114331192894dd80b7d1c8544e3.dist-info/WHEEL
creating '/tmp/tmpml5axba5/tfx_user_code_transform-0.0+72cadd684014286d19d8fc3ce643d86698e6d114331192894dd80b7d1c8544e3-py3-none-any.whl' and adding '/tmp/tmp63jpal_d' to it
adding 'data_preparation.py'
adding 'model_creation.py'
adding 'preprocessing.py'
adding 'tfx_user_code_transform-0.0+72cadd684014286d19d8fc3ce643d86698e6d114331192894dd80b7d1c8544e3.dist-info/METADATA'
adding 'tfx_user_code_transform-0.0+72cadd684014286d19d8fc3ce643d86698e6d114331192894dd80b7d1c8544e3.dist-info/WHEEL'
adding 'tfx_user_code_transform-0.0+7

INFO:tensorflow:Assets written to: /ruta/persistente/para/pipeline_outputs/Transform/transform_graph/14/.temp_path/tftransform_tmp/c3c65a69ffae40718b6458847518bbb0/assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 14
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [28]:
# Obtener los datos transformados y el gráfico de transformación
transformed_examples_channel = transform.outputs['transformed_examples']
transform_graph_channel = transform.outputs['transform_graph']




In [29]:
print(transformed_examples_channel)
print(transform_graph_channel)

OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)


## Inspecciona

In [30]:
train_uri = os.path.join(
    transform.outputs['transformed_examples'].get()[0].uri, 
    'Split-train'
)

tfrecord_filenames = [
    os.path.join(train_uri, name)
    for name in os.listdir(train_uri)
]

dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  print(example)


features {
  feature {
    key: "Cover_Type"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "Elevation_scaled_0_1"
    value {
      float_list {
        value: 0.6561561822891235
      }
    }
  }
  feature {
    key: "Hillshade_9am_scaled_minmax"
    value {
      float_list {
        value: 0.8385826945304871
      }
    }
  }
  feature {
    key: "Hillshade_Noon_scaled_zscore"
    value {
      float_list {
        value: 0.6880866289138794
      }
    }
  }
  feature {
    key: "Horizontal_Distance_To_Fire_Points_scaled_zscore"
    value {
      float_list {
        value: -0.6984237432479858
      }
    }
  }
  feature {
    key: "Horizontal_Distance_To_Hydrology_scaled_0_1"
    value {
      float_list {
        value: 0.20204231142997742
      }
    }
  }
  feature {
    key: "Horizontal_Distance_To_Roadways_scaled_minmax"
    value {
      float_list {
        value: 0.6146711707115173
      }
    }
  }
  feature {
    key: "Slope_scaled

## 5 Metadatos de aprendizaje automático

In [31]:
def display_types(types):
  # Helper function to render dataframes for the artifact and execution types
  table = {'id': [], 'name': []}
  for a_type in types:
    table['id'].append(a_type.id)
    table['name'].append(a_type.name)
  return pd.DataFrame(data=table)
    
def display_artifacts(store, artifacts):
  # Helper function to render dataframes for the input artifacts
  table = {'artifact id': [], 'type': [], 'uri': []}
  for a in artifacts:
    table['artifact id'].append(a.id)
    artifact_type = store.get_artifact_types_by_id([a.type_id])[0]
    table['type'].append(artifact_type.name)
    table['uri'].append(a.uri.replace(base_dir, './'))
  return pd.DataFrame(data=table)

def display_properties(store, node):
  # Helper function to render dataframes for artifact and execution properties
  table = {'property': [], 'value': []}
  for k, v in node.properties.items():
    table['property'].append(k)
    table['value'].append(
        v.string_value if v.HasField('string_value') else v.int_value)
  for k, v in node.custom_properties.items():
    table['property'].append(k)
    table['value'].append(
        v.string_value if v.HasField('string_value') else v.int_value)
  return pd.DataFrame(data=table)

def get_one_hop_parent_artifacts(store, artifacts):
  # Get a list of artifacts within a 1-hop of the artifacts of interest
  artifact_ids = [artifact.id for artifact in artifacts]
  executions_ids = set(
      event.execution_id
      for event in store.get_events_by_artifact_ids(artifact_ids)
      if event.type == mlmd.proto.Event.OUTPUT)
  artifacts_ids = set(
      event.artifact_id
      for event in store.get_events_by_execution_ids(executions_ids)
      if event.type == mlmd.proto.Event.INPUT)
  return [artifact for artifact in store.get_artifacts_by_id(artifacts_ids)]

In [32]:
connection_config = context.metadata_connection_config
store = mlmd.MetadataStore(metadata_config)

# All TFX artifacts are stored in the base directory
base_dir = connection_config.sqlite.filename_uri.split('metadata.sqlite')[0]

## 5.1 Acceso a artefactos almacenados

In [33]:
display_types(store.get_artifact_types())

,id,name
0,14,Examples
1,16,ExampleStatistics
2,18,Schema
3,21,ExampleAnomalies
4,23,TransformGraph
5,24,TransformCache


## 5.2 Seguimiento de artefactos

In [34]:
schema_arifacts = store.get_artifacts_by_type("Schema")
display_artifacts(store, schema_arifacts)

,artifact id,type,uri
0,3,Schema,./pipeline_outputs/SchemaGen/schema/3
1,4,Schema,./pipeline_outputs/ImportSchemaGen/schema/4
2,10,Schema,./pipeline_outputs/Transform/pre_transform_schema/7
3,12,Schema,./pipeline_outputs/Transform/post_transform_schema/7
4,17,Schema,./pipeline_outputs/SchemaGen/schema/10
5,18,Schema,./pipeline_outputs/ImportSchemaGen/schema/11
6,24,Schema,./pipeline_outputs/Transform/pre_transform_schema/14
7,26,Schema,./pipeline_outputs/Transform/post_transform_schema/14


In [35]:
artifacts = schema_arifacts[-1]
display_properties(store, artifacts)

,property,value
0,tfx_version,1.15.1
1,producer_component,Transform
2,name,post_transform_schema:2025-03-03T03:45:16.810987


## 5.3 Obtener artefactos principales

In [36]:
parent_artifacts = get_one_hop_parent_artifacts(store, [artifacts])
display_artifacts(store, parent_artifacts)

,artifact id,type,uri
0,15,Examples,./pipeline_outputs/CsvExampleGen/examples/8
1,18,Schema,./pipeline_outputs/ImportSchemaGen/schema/11
